# Введение

Выполнил: Тихонов Сергей ИАД-5 🤯

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

In [1]:
import pandas as pd
import numpy as np
import math

## Загрузка и предобработка данных

Загрузим данные и проведем предобраотку данных как на семинаре.

In [2]:
!wget -q -N https://www.dropbox.com/s/z8syrl5trawxs0n/articles.zip?dl=0 -O articles.zip
!unzip -o -q articles.zip

In [3]:
articles_df = pd.read_csv('articles/shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [4]:
interactions_df = pd.read_csv('articles/users_interactions.csv')
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [5]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [6]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

interactions_df['eventStrength'] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [7]:
users_interactions_count_df = (
    interactions_df
    .groupby(['personId', 'contentId'])
    .first()
    .reset_index()
    .groupby('personId').size())
print('# users:', len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
print('# users with at least 5 interactions:',len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [8]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [9]:
print('# interactions before:', interactions_df.shape)
print('# interactions after:', interactions_from_selected_users_df.shape)

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [10]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId']).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(['personId', 'contentId'])
)
interactions_full_df['last_timestamp'] = (
    interactions_from_selected_users_df
    .groupby(['personId', 'contentId'])['timestamp'].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [11]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))

interactions_train_df

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093
...,...,...,...,...
39099,997469202936578234,9112765177685685246,2.0,1472479493
39100,998688566268269815,-1255189867397298842,1.0,1474567164
39101,998688566268269815,-401664538366009049,1.0,1474567449
39103,998688566268269815,6881796783400625893,1.0,1474567675


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [12]:
interactions = (
    interactions_train_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={'contentId': 'true_train'})
    .set_index('personId')
)

interactions['true_test'] = (
    interactions_test_df
    .groupby('personId')['contentId'].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), 'true_test'] = [
    list() for x in range(len(interactions.loc[pd.isnull(interactions.true_test), 'true_test']))]

interactions.head(2)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."


## Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [13]:
!pip install lightfm
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

     |████████████████████████████████| 317kB 3.7MB/s 
  Created wheel for lightfm: filename=lightfm-1.16-cp37-cp37m-linux_x86_64.whl size=705345 sha256=6b0d38b41d32ef5f77a35b9a7cf3ee485e10ab3f46662725e268fe4c045a2115
  Stored in directory: /root/.cache/pip/wheels/c6/64/d4/673c7277f71ac4c5ad4835b94708c01b653ef2d3aa78ef20aa
Successfully built lightfm


## Задание 1. (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [14]:
# В последующих заданиях возникает проблема: чтобы посчитать качество на тесте, матрица должна быть такого же размера, что и трейн.
# У нас они изначально разного размера. Для этого создадим новые нулевые матрицы и перезаполним их с учетом информации из трейна и теста.
ratings = pd.pivot_table(
interactions_full_df,
values='eventStrength',
index='personId',
columns='contentId').fillna(0)

ratings_train = pd.pivot_table(
interactions_train_df,
values='eventStrength',
index='personId',
columns='contentId').fillna(0)

ratings_test = pd.pivot_table(
interactions_test_df,
values='eventStrength',
index='personId',
columns='contentId').fillna(0)

In [15]:
data_train_df = pd.DataFrame(0, index=ratings.index, columns=ratings.columns)
for person in ratings_train.index:
  for item in ratings_train.columns:
    if ratings_train[item][person] != 0:
      data_train_df[item][person] = ratings_train[item][person]

data_test_df = pd.DataFrame(0, index=ratings.index, columns=ratings.columns)
for person in ratings_test.index:
  for item in ratings_test.columns:
    if ratings_test[item][person] != 0:
      data_test_df[item][person] = ratings_test[item][person]   

In [16]:
from scipy.sparse import csr_matrix

train_data = csr_matrix(data_train_df)
test_data = csr_matrix(data_test_df)

In [17]:
# Теперь train и test одного размера!
print(repr(train_data)) 
print(repr(test_data))

<1140x2984 sparse matrix of type '<class 'numpy.longlong'>'
	with 29329 stored elements in Compressed Sparse Row format>
<1140x2984 sparse matrix of type '<class 'numpy.longlong'>'
	with 9777 stored elements in Compressed Sparse Row format>


## Задание 2. (1 балл)

Обучите модель LightFM с `loss='warp'` и посчитайте *precision@10* на тесте.

In [18]:
np.random.seed(42)
model = LightFM(loss='warp', random_state=3)
model.fit(train_data, epochs=20)

In [19]:
print("Train precision: %.5f" % precision_at_k(model, train_data, k=10).mean())
print("Test precision: %.5f" % precision_at_k(model, test_data, train_interactions=train_data, k=10).mean())

Train precision: 0.22419
Test precision: 0.00652


## Задание 3. (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss='warp'` и посчитайте precision@10 на тесте.

In [84]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm_notebook

tfidf_vec = TfidfVectorizer(min_df=0.1)

In [85]:
features = []
articles_df.index = articles_df.contentId

for item in tqdm_notebook(ratings.columns):
  try:
    txt = articles_df.loc[item].text
  except:
    txt = 'unknown'
  features.append(txt)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


In [86]:
features_tfidf = tfidf_vec.fit_transform(features)

In [87]:
model = LightFM(loss='warp', random_state=50)
model.fit(train_data, item_features=features_tfidf, epochs=20)

In [88]:
print("Train precision: %.5f" % precision_at_k(model, train_data, item_features=features_tfidf, k=10).mean())
print("Test precision: %.5f" % precision_at_k(model, test_data, train_interactions=train_data, item_features=features_tfidf, k=10).mean())

Train precision: 0.10333
Test precision: 0.00906


## Задание 4. (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [25]:
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
 
stop_languages_words = set(stopwords.words('english') + stopwords.words('portuguese') + stopwords.words('spanish') + list(string.punctuation))
lemmatizer = WordNetLemmatizer()

tfidf_vec = TfidfVectorizer(min_df=0.1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [26]:
def process_text(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(text.lower()) if word not in stop_languages_words])

In [89]:
features_preprocessed = []

for item in tqdm_notebook(ratings.columns):
  try:
    txt = articles_df.loc[item].text
  except:
    txt = 'unknown'
  txt_preprocessed = process_text(txt)
  features_preprocessed.append(txt_preprocessed)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [90]:
features_tfidf_preprocessed = tfidf_vec.fit_transform(features_preprocessed)

In [29]:
model = LightFM(loss='warp', random_state=45)
model.fit(train_data, item_features=features_tfidf_preprocessed, epochs=20)

In [30]:
print("Train precision: %.5f" % precision_at_k(model, train_data, item_features=features_tfidf_preprocessed, k=10).mean())
print("Test precision: %.5f" % precision_at_k(model, test_data, train_interactions=train_data, item_features=features_tfidf_preprocessed, k=10).mean())

Train precision: 0.10567
Test precision: 0.01212


Улучшилось ли качество предсказания?

Качество немного улучшилось, но изменения небольшие. Это можно объяснить тем, что текст многих статей не только на английском языке (много статей на испанском), хотя и в них удалось удалить "стоп слова" и "лемматизировать" некоторые слова. К тому же, поскольку датасет с научными / научно-популярными статьями, то тематики в принципе смещены (блокчейн / криптовалюты / диплернинг), а такие тексты сложнее лемматизировать.
Выход: подбирать гиперпараметры!

## Задание 5. (2 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [200]:
model = LightFM(loss='warp', no_components=21, learning_rate=0.035, random_state=8)
model.fit(train_data, item_features=features_tfidf_preprocessed)

print("Test precision: %.5f" % precision_at_k(model, test_data, train_interactions=train_data, item_features=features_tfidf_preprocessed, k=10).mean())

Test precision: 0.01273


## Бонусное задание. (3 балла)

Выше мы использовали достаточно простое представление текста статьи в виде TF-IDF. В этом задании Вам нужно представить текст статьи (можно вместе с заголовком) в виде эмбеддинга полученного с помощью рекуррентной сети или трансформера (можно использовать любую предобученную модель, которая Вам нравится). Обучите модель с ипользованием этих эмеддингов и сравните результаты с предыдущими.

**Способ 1**

Для начала, попробуем посмотреть качество используя обыкновенные предобученные Word2Vec. Каждое слово статьи (в нашем случае их 2984, но 8 из них пустые) будет переводиться в векторное представление, после чего полученные эмбединги усредняются.

In [32]:
import gensim
size = 500
w2v = gensim.models.Word2Vec(features_preprocessed, size=size, window=5)

def document_vector_mean(doc):
    doc = [word for word in word_tokenize(doc) if word in w2v.wv.vocab]
    try:
      return np.mean(w2v[doc], axis=0)
    except:
      return np.zeros(size)

features_w2v_preprocessed = []
for elem in features_preprocessed:
    features_w2v_preprocessed.append(document_vector_mean(elem))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


In [33]:
features_w2v_preprocessed_csr = csr_matrix(features_w2v_preprocessed)

model = LightFM(loss='warp', no_components=15, learning_rate=0.03, random_state=2)
model.fit(train_data, item_features=features_w2v_preprocessed_csr)

print("Test precision: %.5f" % precision_at_k(model, test_data, train_interactions=train_data, item_features=features_w2v_preprocessed_csr, k=10).mean())

Test precision: 0.00295


Итог: качество на тесте хуже, чем при использовании обыкновенной модели без фичей. Это могло произойти из-за того, что мы попытались обучить Word2Vec на относительно небольшом корпусе текстов. 

**Способ 2**

Попробуем теперь воспользоваться предобученными Трансформерами. 

In [ ]:
pip install -U sentence-transformers

In [35]:
from sentence_transformers import SentenceTransformer
# Попробуем модель distilbert отсюда https://www.sbert.net/docs/pretrained_models.html
model = SentenceTransformer('msmarco-distilbert-base-v3') 

In [36]:
sentence_embeddings = model.encode(features_preprocessed) # перевод занял 35 минут :)

In [38]:
features_transformed_csr = csr_matrix(sentence_embeddings)

model = LightFM(loss='warp', no_components=19, learning_rate=0.03, item_alpha=0.7, random_state=2)
model.fit(train_data, item_features=features_transformed_csr)

print("Test precision: %.5f" % precision_at_k(model, test_data, train_interactions=train_data, item_features=features_transformed_csr, k=10).mean())

Test precision: 0.00815


Итог: использование мощного трансформера позволило превысить качество у всех предыдущих алгоритмов, однако не позволило преодолеть качество на TF-IDF векторах, обученных на данном корпусе текстом с подобранными гиперпараметрами.